In [1]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
# Read the data
X_full = pd.read_csv('Pokemon.csv')

In [3]:
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['Legendary'], inplace=True)
y = X_full.Legendary
X_full.drop(['Legendary'], axis=1, inplace=True)

In [4]:
# Break off validation set from training data
X_train_full, X_test_full, y_train, y_test = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=1)

In [5]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]


In [6]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

In [7]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [8]:
X_train.head()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
797,720,600,80,110,60,150,130,70,6
411,375,420,60,75,100,55,80,50,3
0,1,318,45,49,49,65,65,45,1
318,294,360,84,71,43,71,43,48,3
555,496,413,60,60,75,60,75,83,5


In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [25]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

In [26]:
# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [27]:
# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [28]:
# Define model 1
model1 = RandomForestRegressor(n_estimators=100, random_state=0)

In [29]:
#Define model 2
model2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

In [30]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline1 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model1)
                             ])

my_pipeline2 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model2)
                             ])

In [31]:
# Preprocessing of training data, fit model 
my_pipeline1.fit(X_train, y_train)
my_pipeline2.fit(X_train, y_train)

C:\Users\Abhishek Mohanty\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:01:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='constant', verbose=0), ['#', 'Total', 'HP', 'Attack', 'Defen...lpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1))])

In [32]:
# Preprocessing of validation data, get predictions
preds1 = my_pipeline1.predict(X_test)
preds2 = my_pipeline2.predict(X_test)

In [34]:
# Evaluate the model
score1 = mean_absolute_error(y_test, preds1)
score2 = mean_absolute_error(y_test, preds2)
print('MAE: (MODEL - RandomForestRegressor) ', score1)
print('MAE: (MODEL - ExtremeGradientBoost) ', score2)

MAE: (MODEL - RandomForestRegressor)  0.0480625
MAE: (MODEL - ExtremeGradientBoost)  0.047055732
